# TRABAJO:

### Realice un programa que capture un video y desarrolle un procedimiento de detección de movimiento de una persona por sustracción de fondo e identifique la persona en movimiento con un rectángulo, ubicando a la misma en todo momento. Adicionalmente, utilice una prenda para eliminar una parte del cuerpo de la persona y que, en lugar de dicha parte, se aprecie el fondo de la escena. Pero si en la parte posterior de la extremidad oculta, se encuentra la persona, debe visualizarse la sección del cuerpo que se encuentra ocluida. Quiere decir, que no debe aparecer el fondo al interior de la persona

In [ ]:
import cv2
import time
import numpy as np
from matplotlib import colors
from matplotlib import pyplot as plt
kernel = np.ones((5,5),np.uint8)

#-------------------------------------------------------------------------------------------------------------------

#camara = cv2.VideoCapture(0) #Captura de video con la camara del computador
cap = cv2.VideoCapture(0) #Captura de video con la camara del Web LifeCam VX-1000
#camara = cv2.VideoCapture('stres.mp4') #Video ya existente
_,frame1=cap.read()
#fondo = None #Obtenemos el primer Frame como "VACIO" obteniendo asi el fondo con el que comparar

fondo = None

while(1):
    #-------------------------------------------------------------------------------------------------------------------   
    # Take each frame
    _, frame = cap.read()
      
    # define range of gree color
    lower_green = np.array([25,25,61])
    upper_green = np.array([43,38,107])
   
    # Threshold the HSV image to get only blue colors
    mask = cv2.inRange(frame, lower_green, upper_green)
    opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    x,y,w,h = cv2.boundingRect(opening)
    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),4)
    resu1 = cv2.bitwise_and(frame1,frame1, mask=mask)
    resu2 = cv2.bitwise_or(frame,resu1)
    #-------------------------------------------------------------------------------------------------------------------   
    
    nueva = frame.copy()
    gris = cv2.cvtColor(nueva, cv2.COLOR_BGR2GRAY)
    gris = cv2.GaussianBlur(gris, (21, 21), 0)
    
    if fondo is None:
        fondo = gris
        continue
    
    resta = cv2.absdiff(fondo, gris)
    umbral = cv2.threshold(resta, 25, 255, cv2.THRESH_BINARY)[1]
    umbral = cv2.dilate(umbral, None, iterations=2)
    contornosimg = umbral.copy()
    
    contornos,_ = cv2.findContours(contornosimg,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contornos:
        if cv2.contourArea(c) < 20:
            continue
        (x, y, w, h) = cv2.boundingRect(c) 
        cv2.rectangle(nueva, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
    #-------------------------------------------------------------------------------------------------------------------   
    cv2.imshow('frame , codigo color',frame)
    cv2.imshow('mask',mask)
    cv2.imshow('resul1',resu1)
    cv2.imshow('resul2',resu2)
    
    cv2.imshow("Camara", nueva) #mostramos el video de la camara
    cv2.imshow("Umbral", umbral)#mostramos la mascra del umbral
    cv2.imshow("Resta", resta)#mostramos la mascara de la resta
    cv2.imshow("Contorno", contornosimg)#mostramos los contornos
    
    #-------------------------------------------------------------------------------------------------------------------   
    tecla = cv2.waitKey(27) & 0xFF
    if tecla == 27: #para salir del while
        break

#-------------------------------------------------------------------------------------------------------------------
cap.release()    #libero la camara para otros programas
cv2.destroyAllWindows() #cierro las ventanas abiertas